In [ ]:
import torch
import transformers
import sentence_transformers
import sklearn
import numpy
import scipy

print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"sentence-transformers: {sentence_transformers.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy: {numpy.__version__}")
print(f"scipy: {scipy.__version__}")

from dotenv import load_dotenv
import os

load_dotenv()  # 같은 폴더에 있는 .env 로드

/opt/anaconda3/envs/skala/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.2.2
transformers: 4.25.1
sentence-transformers: 2.2.2
scikit-learn: 1.3.2
numpy: 1.24.4
scipy: 1.10.1


In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 1단계: 문장 임베딩
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

reviews = [
    "배송이 빠르고 제품도 좋아요.",
    "품질이 기대 이상입니다!",
    "생각보다 배송이 오래 걸렸어요.",
    "배송은 느렸지만 포장은 안전했어요.",
    "아주 만족스러운 제품입니다."
]

embeddings = model.encode(reviews)

In [ ]:
import psycopg2
from pgvector.psycopg2 import register_vector

# 2단계: PostgreSQL 테이블 생성
conn = psycopg2.connect(
    host="localhost",          
    port=5432,                
    database="postgres",    
    user="postgres",       
    password=os.getenv("PG_PASSWORD"),  # 환경변수에서 불러옴 
)

register_vector(conn)  # 벡터 변환 활성화
cur = conn.cursor()

# DB 초기화 (기존 테이블 삭제 후 재생성)
dim = model.get_sentence_embedding_dimension()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
cur.execute("DROP TABLE IF EXISTS review_vectors;")  # 테이블 완전 삭제
cur.execute(f"""
    CREATE TABLE review_vectors (
        id SERIAL PRIMARY KEY,
        review TEXT,
        embedding vector({dim})
    );
""")
conn.commit()

In [5]:
# 3단계: 벡터 저장
for review, emb in zip(reviews, embeddings):
    cur.execute(
        "INSERT INTO review_vectors (review, embedding) VALUES (%s, %s)",
        (review, np.array(emb, dtype=np.float32)) 
    )
conn.commit()

In [ ]:
# 4단계: 유사도 검색
query = "배송이 느렸어요"
query_vec = model.encode([query])[0].astype(np.float32)

print("\n유사도 검색 결과:")
cur.execute(
    """
    SELECT review, embedding <=> %s AS cosine_distance
    FROM review_vectors
    ORDER BY embedding <=> %s
    LIMIT 3;
    """,
    (query_vec, query_vec)
)
for review, dist in cur.fetchall():
    print(f"코사인거리: {dist:.4f} | 리뷰: {review}")


유사도 검색 결과:
코사인거리: 0.0783 | 리뷰: 배송이 빠르고 제품도 좋아요.
코사인거리: 0.0990 | 리뷰: 배송은 느렸지만 포장은 안전했어요.
코사인거리: 0.1253 | 리뷰: 생각보다 배송이 오래 걸렸어요.


In [7]:
# 5. 마무리
cur.close()
conn.close()